In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.datasets import load_wine

### Load the dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Tuwaiq/Week 3/3- Advanced Machine Learning/2- Ensemble learning Methods and Regularization/LAB/Dataset/WineDataset.csv')
df.head()

,Title,Description,Price,Capacity,Grape,Secondary Grape Varieties,Closure,Country,Unit,Characteristics,Per bottle / case / each,Type,ABV,Region,Style,Vintage,Appellation
0,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,£9.99 per bottle,75CL,Tempranillo,NaN,Natural Cork,Spain,10.5,"Vanilla, Blackberry, Blackcurrant",per bottle,Red,ABV 14.00%,NaN,Rich & Juicy,NV,NaN
1,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. It’s...,£15.99 per bottle,75CL,Chardonnay,NaN,Natural Cork,USA,10.1,"Vanilla, Almond, Coconut, Green Apple, Peach, ...",per bottle,White,ABV 13.50%,California,Rich & Toasty,2021,Napa Valley
2,"Oyster Bay Sauvignon Blanc 2022, Marlborough",Oyster Bay has been an award-winning gold-stan...,£12.49 per bottle,75CL,Sauvignon Blanc,NaN,Screwcap,New Zealand,9.8,"Tropical Fruit, Gooseberry, Grapefruit, Grass,...",per bottle,White,ABV 13.00%,Marlborough,Crisp & Zesty,2022,NaN
3,Louis Latour Mâcon-Lugny 2021/22,We’ve sold this wine for thirty years – and fo...,£17.99 per bottle,75CL,Chardonnay,NaN,Natural Cork,France,10.1,"Peach, Apricot, Floral, Lemon",per bottle,White,ABV 13.50%,Burgundy,Ripe & Rounded,2022,Macon
4,Bread & Butter 'Winemaker's Selection' Pinot N...,Bread & Butter is that thing that you can coun...,£15.99 per bottle,75CL,Pinot Noir,NaN,Natural Cork,USA,10.1,"Smoke, Black Cherry, Cedar, Raspberry, Red Fruit",per bottle,Red,ABV 13.50%,California,Smooth & Mellow,2021,Napa Valley


In [5]:
df['Price'].nunique()

128

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Title                      1290 non-null   object 
 1   Description                1286 non-null   object 
 2   Price                      1290 non-null   object 
 3   Capacity                   1290 non-null   object 
 4   Grape                      1275 non-null   object 
 5   Secondary Grape Varieties  488 non-null    object 
 6   Closure                    1279 non-null   object 
 7   Country                    1284 non-null   object 
 8   Unit                       1281 non-null   float64
 9   Characteristics            1253 non-null   object 
 10  Per bottle / case / each   1290 non-null   object 
 11  Type                       1285 non-null   object 
 12  ABV                        1281 non-null   object 
 13  Region                     1124 non-null   objec

In [52]:
df.describe()

,Unit,ABV,Vintage
count,1273.000000,1273.000000,1273.000000
mean,10.161822,13.419324,1759.657502
std,2.282994,1.799656,676.952385
min,0.000000,0.500000,0.000000
25%,9.400000,12.500000,2018.000000
50%,10.100000,13.500000,2020.000000
75%,10.500000,14.000000,2022.000000
max,39.000000,40.000000,2023.000000


### Preprocess the data (if necessary)

#### Null values

In [4]:
df.isna().sum()

Title                          0
Description                    4
Price                          0
Capacity                       0
Grape                         15
Secondary Grape Varieties    802
Closure                       11
Country                        6
Unit                           9
Characteristics               37
Per bottle / case / each       0
Type                           5
ABV                            9
Region                       166
Style                         78
Vintage                        7
Appellation                  646
dtype: int64

In [5]:
df.drop(columns=['Appellation' , 'Description', 'Secondary Grape Varieties', 'Region', 'Characteristics',
                 'Per bottle / case / each' , 'Title'] , axis=1 , inplace=True)


In [6]:
df['Grape'].fillna(df['Grape'].mode()[0] , inplace=True)

df['Closure'].fillna(df['Closure'].mode()[0] , inplace=True)

df['Style'].fillna(df['Style'].mode()[0] , inplace=True)

df = df.dropna(subset=['Type', 'Unit', 'ABV'])

In [7]:
df.isna().sum()

Price       0
Capacity    0
Grape       0
Closure     0
Country     0
Unit        0
Type        0
ABV         0
Style       0
Vintage     0
dtype: int64

#### inconsestane data

In [14]:
df['Vintage'] = df['Vintage'].str.replace('2021/2022', '2021')
df['Vintage'] = df['Vintage'].str.replace('2012/2015', '2013')
df['Vintage'] = df['Vintage'].str.replace('2020/2022', '2021')
df['Vintage'] = df['Vintage'].str.replace('2019/20', '2019')
df['Vintage'] = df['Vintage'].str.replace('22020/2021', '2020')
df['Vintage'] = df['Vintage'].str.replace('2020/21', '2021')
df['Vintage'] = df['Vintage'].str.replace('2016/7', '2017')
df['Vintage'] = df['Vintage'].str.replace('2021/22', '2021')
df['Vintage'] = df['Vintage'].str.replace('2020/2021', '2020')
df['Vintage'] = df['Vintage'].str.replace('NV', '0')

df['Capacity'] = df['Capacity'].str.replace('ML', '')
df['Capacity'] = df['Capacity'].str.replace('TR', '')
df['Capacity'] = df['Capacity'].str.replace('L', '')
df['Capacity'] = df['Capacity'].str.replace('CL', '')
df['Capacity'] = df['Capacity'].str.replace('C', '')

df['Price'] = df['Price'].str.replace('per bottle', '')
df['Price'] = df['Price'].str.replace('£', '')
df['Price'] = df['Price'].str.replace('each', '')

#### Chnage data type

In [15]:
df['Vintage'] = df['Vintage'].astype(float)

In [10]:
df['Price'] = df['Price'].astype(float)

In [16]:
df['Capacity'] = df['Capacity'].astype(float)

In [17]:
df['ABV'] = df['ABV'].str.replace('%', '')
df['ABV'] = df['ABV'].str.replace('ABV', '').astype(float)

### Split the data

In [18]:
X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Scalling - Encoding

In [19]:
X_train.head()

,Capacity,Grape,Closure,Country,Unit,Type,ABV,Style,Vintage
998,75.0,Merlot,Natural Cork,France,10.5,Red,14.0,Savoury & Full Bodied,2019.0
424,75.0,Chardonnay,Natural Cork,France,10.1,White,13.5,Rich & Toasty,2022.0
793,75.0,Tinta Roriz,Screwcap,Portugal,10.5,Red,14.0,Rich & Juicy,2020.0
932,75.0,Cabernet Sauvignon,Natural Cork,France,10.9,Red,14.5,Bold & Spicy,2021.0
96,75.0,Sauvignon Blanc,Screwcap,France,9.8,White,13.0,Fresh & Elegant,2022.0


In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

num_pip = make_pipeline(MinMaxScaler())

scaler = StandardScaler()
cat_pip_one_hot = make_pipeline(OneHotEncoder(handle_unknown='ignore'))


pre_pip = ColumnTransformer([
    ('cat_one_hot', cat_pip_one_hot, ['Grape',  'Closure' , 'Country', 'Type',
                                      'Style']),
    ('num_scaling' , scaler , ['Capacity' ,'Unit', 'ABV', 'Vintage' ])
    ])


In [22]:
X_train = pre_pip.fit_transform(X_train)
X_test = pre_pip.transform(X_test)

### Implement a stacking model

In [38]:
base_models = [
    ('decision_tree', DecisionTreeRegressor(random_state=42)),
    ('svr', SVR()),
    ('knn', KNeighborsRegressor()),
    ('random_forest', RandomForestRegressor(random_state=42))
]

meta_model = LinearRegression()

In [39]:
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model, cv=4)


stacking_regressor.fit(X_train, y_train)

StackingRegressor(cv=4,
                  estimators=[('decision_tree',
                               DecisionTreeRegressor(random_state=42)),
                              ('svr', SVR()), ('knn', KNeighborsRegressor()),
                              ('random_forest',
                               RandomForestRegressor(random_state=42))],
                  final_estimator=LinearRegression())

### Evaluate the model performance

In [50]:
y_pred = stacking_regressor.predict(X_test)
y_pred

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


Mean Absolute Error: 11.575932055018736
